<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/lec_title.png?raw=true" alt="2019年度ゲノム情報解析入門" height="100px" align="middle">

# 機械学習 - 最適値探索 -

In [0]:
# /// 実習前にこのセルを実行してください ///
# データの読み込み
!wget -q -O gene_expression.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/dataset/gene_expression.csv
# 勾配法用モジュール
!wget -q -O gradient_method.py https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/modules/gradient_method.py
# 動画再生用モジュール
!apt-get -q install ffmpeg

## 今回の実習内容

　今回は、最適値の探索方法について学びます。まず、線形回帰を例に、「勾配法」と呼ばれるパラメータ（係数や誤差）の最適値を探索する方法を学びます。また、「グリッドサーチ」と呼ばれる適切なハイパーパラメータ（初期設定値）を見つける方法についても学びます。

1. 勾配法とは？
1. 確率的勾配降下法 Stochastic Gradient Descent
1. 最急降下法 Gradient Descent
1. グリッドサーチ Grid search

---

## 1. 勾配法とは？
　線形回帰モデルは次のような式で表されます。
$$y=\beta_1 x_1 + \beta_2 x_2 + ... + \beta_m x_m + e $$

　前回は、scikit-learnの`LinearRegression`を用いて、説明変数が数個（1,2個）の場合の線形回帰をおこないました。`LinearRegression`は、「最小二乗法」を用いて、残差平方和 $\sum(\hat{y}_i - y_{i})^{2}$ が最小値をとるときの係数$\beta$と誤差$e$ を求めることも学びました。

　最小二乗法は、説明変数が少ない場合、一瞬のうちに解を求めることができます。しかし、説明変数が多くなると、計算量がとてつもなく多くなり、短時間で解が得られなくなります。そのような場合に有効な方法が**勾配法（Gradient method）**です。

　最小二乗法と勾配法はどう違うのか？　最小二乗法は、計算により最小値をピンポイントに見つける方法です。勾配法は、ある値から出発して、係数$\beta$や誤差$e$の値を徐々に更新し、最小値に近づいていく方法です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method.png?raw=true" alt="gradient_method" height="250px">

### 最小二乗法の計算コスト
_※ 数学（行列）の話になるので、読み飛ばしてOKです。_

　$i$行$j$列のトレーニングデータセットの説明変数を行列$X$で表すと、次のようになります。1行は1サンプル分のデータで、各列は説明変数の値です。また、目的変数$y$も$i$行1列の行列で表すことができ、係数$\beta$も$j$行1列の行列で表すことができます。

$$
X = \begin{pmatrix}
x_{11} & x_{12} & x_{12} & ... & x_{1j} \\
x_{21} & x_{22} & x_{22} & ... & x_{2j} \\
x_{31} & x_{32} & x_{32} & ... & x_{3j} \\
 : & : & : & ... & : \\
 x_{i1} & x_{i2} & x_{i2} & ... & x_{ij} \\
\end{pmatrix}, 
y = \begin{pmatrix}
y_{1}  \\
y_{2} \\
y_{3} \\
 :  \\
y_{i} \\
\end{pmatrix},  
\beta = \begin{pmatrix}
\beta_{1}  \\
\beta_{2} \\
\beta_{3} \\
 :  \\
\beta_{j} \\
\end{pmatrix}
$$

　最小二乗法で、残差平方和を最小化する$\hat{\beta}$を求めるとき、実際には次のような行列式を解いています。
$$ \hat{\beta} = (X^{T} X)^{-1} X^{T} y $$

　説明変数$j$が多くなると、上述の式の$(X^{T} X)^{-1}$（逆行列）の計算が大規模計算機を用いても困難になってきます（時間がかかりすぎるか、メモリ上に数値を保持できなくなります）。

　もう少し具体的に言うと、
- $X^{T}$（$j\times i$行列）と$X$（$i\times j$行列）の積$X^{T} X$により、$j\times j$行列ができます。この$j\times j$行列の逆行列を求めることになります。
- LU分解を利用して $j\times j$行列の逆行列を得ようとすると、$j^3$の計算量が必要になります（「[プログラミングのための線形代数](https://www.ohmsha.co.jp/book/9784274065781/)」より）。


### 勾配法のアルゴリズム

　勾配法は、上述したとおり、「ある値から出発して、係数𝛽や誤差𝑒の値を徐々に更新し、最小値に近づいていく方法」です。ここではその方法について、もう少し具体的な話をしましょう。

　次の図のように、下に凸なグラフがあり、現在の$\beta$の値が$\beta_0$だったとします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method_example.png?raw=true" alt="gradient_method_example" height="250px">

- [Question] 目的関数$Cost(\beta)$が最小値となる$\beta_{optimum}$に近づくためには、現在の$\beta$からどのように値を変化させればいいでしょうか？  
  - [Answer] $\beta$が右側に移るような値（$\beta_0$に正の値）を追加すれば、$\beta_{optimum}$側に近づきます。
- [Question] どの程度の正の値を追加すれば良いでしょうか？
  - [Answer] 大きすぎない値を追加すれば、$\beta_{optimum}$に近づきます。

　勾配法では、上のQ&Aのような処理を、次のようにおこなっています。
1. まず、$\beta_0$の「接線の傾き」を調べます。すなわち、$Cost(\beta)$を$\beta$について微分したときの値を調べます。
  $$ \frac{\partial Cost(\beta)}{\partial  \beta} $$

1. その値が負のとき、右方向に移るように$\beta$の値を更新します。その時の更新幅を、傾きの大きさと**学習率$\eta$（イータ; eta）**の積で決定します。
  $$ \beta_{1} := \beta_{0} - \eta \frac{\partial Cost(\beta)}{\partial  \beta} $$

1. この1と2のステップを$\beta$の値が変化しなくなるまで（もしくは、指定の繰り返し数に達するまで）、ひたすら繰り返します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method_algorithm.png?raw=true" alt="gradient_method_algorithm" height="250px">


### おもな勾配法

　おもな勾配法には下記の3つがあります。これらの違いは、勾配の計算をおこなうときに使うサンプルセットが異なっている点です。そのほかのアルゴリズムはほとんど同じです。
- **確率的勾配降下法 Stochastic Gradient Descent**
  - ランダムに選んだ1サンプルのデータを使って勾配の計算をおこない、パラメータを更新する
- **最急降下法（バッチ勾配降下法） Gradient Descent**
  - 全サンプルのデータセットを使って勾配の計算をおこない、パラメータを更新する
- **ミニバッチ勾配降下法 Mini batch Gradient Descent**
  - ランダムに選んだ複数サンプルのデータセットを使って勾配の計算をおこない、パラメータを更新する

　このテキストでは、「確率的勾配降下法」と「最急降下法」の概要と実装方法をみていきます。



### 今回の実習のサンプルデータ

　扱うデータは、前回と同じく、遺伝子発現量と表現型のデータセットです。勾配法の仕組みを学ぶために、説明変数1個と目的変数1個のみを使います。

　次のコードセルを実行して、データを読み込み、説明変数xと目的変数yの値を得てください。実行すると、自動的に散布図まで書かれるようになっています。

<small>_※ 勾配法の仕組みを勉強するだけなので、「トレーニングデータとテストデータの分割」や「モデルの評価」、「新しいデータの予測」は今回おこないません。_</small>





In [0]:
# 説明変数xとして使うデータ
# gene_1からgene_50であれば、どれでもOK（初期値: gene_29）
use_var = "gene_29"  

#===== 以下は変更しないでください =====
# ライブラリ
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler 
# データの読み込み
df = pd.read_csv("gene_expression.csv", sep=",", header=0)
# 変数x, y　（＊重要＊ 説明変数xは正規化をおこなっている）
mms = MinMaxScaler()
x = mms.fit_transform(np.array(df[use_var]).reshape(-1,1))
y = np.array(df["phenotype"])

# グラフ
import matplotlib.pyplot as plt
plt.figure(figsize=[8, 4])
plt.scatter(x, y, color="blue", label="training data") # データ
plt.xlabel("normalized x")  # x軸ラベル
plt.ylabel("y")  # y軸ラベル
plt.legend()         # 凡例
plt.show()

### 参考: 最小二乗法で線形回帰モデルを推定した場合...

　この例を「最小二乗法」を使って線形回帰モデルを推定すると、次の係数$\beta$と誤差$e$が得られます。

In [0]:
# =================== 線形回帰 ===================
from sklearn.linear_model import LinearRegression

# モデル作成・学習
model_lr = LinearRegression()
model_lr.fit(x, y)

# 学習後の係数と誤差を確認
print("b=", model_lr.coef_)
print("e=", model_lr.intercept_)

# 決定係数
print("R2=", model_lr.score(x, y))

# =================== グラフ ===================
import matplotlib.pyplot as plt

# 直線を書くためのデータ
x_line = np.linspace(np.min(x), np.max(x), num=2)
y_line = model_lr.predict(x_line.reshape(-1, 1))

# グラフ
plt.figure(figsize=[8, 4])
plt.scatter(x, y, color="blue", label="training data") # データ
plt.plot(x_line, y_line, color="orange")
plt.xlabel("normalized x")  # x軸ラベル
plt.ylabel("y")  # y軸ラベル
plt.legend()         # 凡例
plt.show()

## 2. 確率的勾配降下法 Stochastic Gradient Descent

### 概要

　**確率的勾配降下法（Stochastic Gradient Descent: SGD）**は、ランダムに（確率的に）選んだ1サンプルを使って勾配計算をおこない、係数$\beta$や誤差$e$などの**パラメータ（Parameter）**を少しずつ更新する方法です。

　このアルゴリズムは、次のとおりです。
1. トレーニング回数（イテレーション数）やパラメータの更新幅（**学習率**）を設定します。
1. パラメータ（係数や誤差）の初期値を設定する。多くの場合、「0」または「ごく小さいランダムな数値（小数点数）」を設定します。
1. 上で設定したトレーニング回数の分だけ、以下の操作を繰り返します。  
  次の(1)-(3)を全データに対しておこないます。  
  (1) データセットからランダムに1サンプル選びます。  
  (2) そのデータを使って、勾配計算をおこないます。  
  (3) 勾配と学習率をかけた値を、パラメータから引き、パラメータ更新をおこないます。
    $$ 更新後パラメータ := 更新前パラメータ - 学習率 \times 勾配$$

 

### 実装

　scikit-learnで「確率的勾配降下法」をおこなうことが可能です。

```python
# 実装方法
from sklearn.linear_model import SGDRegressor
model = SGDRegressor(max_iter=100, eta0=0.1)
model.fit(x, y)
```

In [0]:
# =================== 勾配法 ===================
from sklearn.linear_model import SGDRegressor

# 設定値
n_iter = 100  # 学習回数
eta   = 0.01   # 学習率

# モデル作成・学習
model_gd = SGDRegressor(max_iter=n_iter, eta0=eta, random_state=1, learning_rate="constant")
model_gd.fit(x, y)
print("b=", model_gd.coef_)
print("e=", model_gd.intercept_)

# 決定係数
r2 = model_gd.score(x, y)
print("R2=", r2)

# グラフ
import matplotlib.pyplot as plt

x_line = np.linspace(np.min(x), np.max(x), num=2)
y_line = model_gd.predict(x_line.reshape(-1, 1))

fig = plt.figure(figsize=[8, 4])
plt.scatter(x, y, color="blue") # データ
plt.plot(x_line, y_line, color="orange")
plt.xlabel("normalized x")  # x軸ラベル
plt.ylabel("y")  # y軸ラベル
plt.show()

### 実習1

　次のデータセット（説明変数`X`、目的変数`Y`）について、「確率的勾配降下法」で回帰モデルを作成してください。その際、以下のオプションを設定してください。

- トレーニング回数`max_iter`: 自由に設定してください
- 学習率`eta0`: 自由に設定してください


In [0]:
# /// このセルは実行のみ ///
import numpy as np

# データ(y = -5x + 3 + 誤差)
N = 100
X = 2 * np.random.rand(N, 1)
Y = (-5*X + 3 + np.random.randn(N, 1)).flatten()

# グラフ
import matplotlib.pyplot as plt
plt.scatter(X, Y, color="blue", label="training data") # データ
plt.xlabel("X")  # x軸ラベル
plt.ylabel("Y", rotation=0)  # y軸ラベル
plt.legend()         # 凡例
plt.show()

In [0]:
# /// このセルに追記してください ///
from sklearn.linear_model import SGDRegressor

# --------------- 編集箇所: start ---------------
# 学習前のモデルを作成
model_sgd = 
# データをセットしてモデルを学習
model_sgd

# --------------- 編集箇所: end -----------------

# 係数と誤差、決定係数
print("b=", model_sgd.coef_)
print("e=", model_sgd.intercept_)
print("R2=", model_sgd.score(X, Y))

# 直線を書くためのデータ
X_line = np.linspace(np.min(X), np.max(X), num=2)
Y_line = model_sgd.predict(X_line.reshape(-1, 1))

# 描画
import matplotlib.pyplot as plt
plt.scatter(X, Y, color="blue", label="training data") # データ
plt.plot(X_line, Y_line, color="orange", label="model") # 直線
plt.xlabel("X")  # x軸ラベル
plt.ylabel("Y", rotation=0)  # y軸ラベル
plt.legend()         # 凡例
plt.show()

#### 解答例

In [0]:
# /// このセルに追記してください ///
from sklearn.linear_model import SGDRegressor

# --------------- 編集箇所: start ---------------
# 学習前のモデルを作成
model_sgd = SGDRegressor(max_iter=100, eta0=0.1)
# データをセットしてモデルを学習
model_sgd.fit(X, Y)

# --------------- 編集箇所: end -----------------

# 係数と誤差、決定係数
print("b=", model_sgd.coef_)
print("e=", model_sgd.intercept_)
print("R2=", model_sgd.score(X, Y))

# 直線を書くためのデータ
X_line = np.linspace(np.min(X), np.max(X), num=2)
Y_line = model_sgd.predict(X_line.reshape(-1, 1))

# 描画
import matplotlib.pyplot as plt
plt.scatter(X, Y, color="blue", label="training data") # データ
plt.plot(X_line, Y_line, color="orange", label="model") # 直線
plt.xlabel("X")  # x軸ラベル
plt.ylabel("Y", rotation=0)  # y軸ラベル
plt.legend()         # 凡例
plt.show()

### 学習経過を記録する

　上で使用した`fit`は、全工程を一通りおこなうものです。途中経過を記録したい場合、`partial_fit`を使って、次のようなコードを書きます。

1. 学習前のモデルを作る
2. ループ構文で、以下を任意回数繰り返す
  1. サンプル順序をシャッフルする（サンプルをランダムに並び替える）
  1. ループ構文で、1サンプルずつ取り出す。
  1. 取り出した1サンプルデータを `partial_fit`に渡して、モデルの学習をおこなう
  1. 全サンプルの学習を終えたら、パラメータなどを記録する



In [0]:
# =================== 勾配法 ===================
from copy import deepcopy
from sklearn.linear_model import SGDRegressor

# 設定値
n_iter = 100  # 学習回数
eta   = 0.01   # 学習率

# 記録用リスト
log_coef = []      # 係数（傾き）
log_intercept = [] # 誤差(切片)
log_cost = []      # 残差二乗和

# モデル作成
model_gd = SGDRegressor(eta0=eta, random_state=1, learning_rate="constant")

# 設定数の学習を繰り返す
for iteration in range(n_iter):
  # トレーニングデータをランダムに並べ替える
  r = np.random.permutation(len(y))

  # 1サンプルずつ学習する
  for i in range(0, len(y), 1):
    idx = r[i:i+1]  # サンプルのインデックスを取得
    x_i = x[idx]       # サンプルのデータを取得
    y_i = y[idx]
    # 学習
    model_gd.partial_fit(x_i, y_i)
  
  # 記録
  log_coef.append(deepcopy(model_gd.coef_))
  log_intercept.append(model_gd.intercept_)
  cost = ((model_gd.predict(x) - y)**2).sum() / 2.0 / len(y)  # コスト（残差二乗和）の計算
  log_cost.append(cost)

# 学習後の係数と誤差を確認
print("b=", model_gd.coef_)
print("e=", model_gd.intercept_)
# 決定係数
print("R2=", model_gd.score(x, y))

　以下のコードセルを実行すると、モデルが学習されていく過程を動画でみれます。

<small>*※ 学習過程のデータを動画用データに変換するコードは、こちらに記述しています: [gradient_method.py](https://github.com/CropEvol/lecture/blob/master/textbook_2019/modules/gradient_method.py)の`plot_reg`関数*</small>

In [0]:
# =================== グラフ ===================
import matplotlib.pyplot as plt
from matplotlib import animation, rc, rcParams
from IPython.display import HTML
from gradient_method import plot_reg
rcParams['animation.embed_limit'] = 2**128

# パラメータのログを取得
b_ = log_coef
e_ = log_intercept
c_ = log_cost

# 動画実行
fig = plt.figure(figsize=[16, 4])
plt.close()
frames = plot_reg(fig, x, y, b_, e_, c_, n_frames=100)
ani = animation.ArtistAnimation(fig, frames, interval=100, blit=True)
# ani.save('gradient_descent.mp4', writer="ffmpeg")
rc('animation', html='jshtml')
ani

## 3. 最急降下法 Gradient Descent

### 概要

　「最急降下法」は、全サンプルで勾配計算をおこない、その平均値を使って、係数$\beta$や誤差$e$などの**パラメータ（Parameter）**を更新する方法です。

　このアルゴリズムは、次のとおりです。
1. トレーニング回数（イテレーション数）やパラメータの更新幅（**学習率**）を設定します。
1. パラメータ（係数や誤差）の初期値を設定する。多くの場合、「0」または「ごく小さいランダムな数値（小数点数）」を設定します。
1. 上で設定したトレーニング回数の分だけ、以下の操作を繰り返します。   
  (1) 各サンプルのデータを使って勾配を計算し、その平均値を求めます。  
  (2) 勾配の平均値と学習率をかけた値を、パラメータから引き、パラメータ更新をおこないます。
    $$ 更新後パラメータ := 更新前パラメータ - 学習率 \times \frac{1}{N}\sum 勾配$$


### 実装
　scikit-learnライブラリに「最急降下法」用の関数はありません。自身でコーディングする必要があります（例: [gradient_method.py](https://github.com/CropEvol/lecture/blob/master/textbook_2019/modules/gradient_method.py)の`GradientDescent`）。

　ここでは、「確率的勾配降下法」用の`SGDRegressor`の`partial_fit`を利用して、「最急降下法」をおこないます。

1. 学習前のモデルを作成する `SGDRegressor`
1. データセット全体に対して、`partial_fit`を使い、モデルの学習をおこなう
1. 2を繰り返す


```python
# 実装方法
from sklearn.linear_model import SGDRegressor
model = SGDRegressor()
for i in range(トレーニング回数):
  model.partial_fit(x, y)
```

<small>*※ この`SGDRegressor`の`partial_fit`を使った方法は、実際のところ1サンプルごとにパラメータ更新をおこなっており、真の「最急降下法」（全データにおける勾配の平均値でパラメータ更新）ではありません。しかし、「最急降下法」とほとんど変わりない学習過程を観察できるため、ここではこの方法を使っています。*</small>




In [0]:
# =================== 勾配法 ===================
from copy import deepcopy
from sklearn.linear_model import SGDRegressor

# 設定値
n_iter = 100  # 学習回数
eta   = 0.01   # 学習率

# 記録用リスト
log_coef = []      # 係数（傾き）
log_intercept = [] # 誤差(切片)
log_cost = []      # 残差二乗和

# モデル作成
model_gd = SGDRegressor(eta0=eta, random_state=1, learning_rate="constant")

# 設定数の学習を繰り返す
for iteration in range(n_iter):
  #　学習
  model_gd.partial_fit(x, y)
  # 記録
  log_coef.append(deepcopy(model_gd.coef_))
  log_intercept.append(model_gd.intercept_)
  cost = ((model_gd.predict(x) - y)**2).sum() / 2.0 / len(y)  # コストの計算
  log_cost.append(cost)

# 学習後の係数と誤差を確認
print("b=", model_gd.coef_)
print("e=", model_gd.intercept_)
# 決定係数
print("R2=", model_gd.score(x, y))

# =================== グラフ ===================
import matplotlib.pyplot as plt
from matplotlib import animation, rc, rcParams
from IPython.display import HTML
from gradient_method import plot_reg
rcParams['animation.embed_limit'] = 2**128

# パラメータのログを取得
b_ = log_coef
e_ = log_intercept
c_ = log_cost

# 動画実行
fig = plt.figure(figsize=[16, 4])
plt.close()
frames = plot_reg(fig, x, y, b_, e_, c_, n_frames=100)
ani = animation.ArtistAnimation(fig, frames, interval=100, blit=True)
# ani.save('gradient_descent.mp4', writer="ffmpeg")
rc('animation', html='jshtml')
ani

### 「確率的勾配降下法」と「最急降下法」の比較

|| 確率的勾配降下法 | 最急降下法 |
|---:|---:|---:|
| パラメータ更新 | 1サンプル毎 | データセット毎 |
| 計算量 | 一定 | サンプル数に依存して多くなる |
| 解周辺に辿り着く早さ | 早い | サンプル数に依存して遅くなる |
| 1サンプル（外れ値）の影響 | 影響を受けやすい | 平均値を使うので影響が緩和される |
| パラメータ更新の方向<br>（学習率が低い場合） | 総じて目標の解の方向に更新されるが、<br>ときどき反対方向にも更新される | 一方向に更新される |
| 局所解 | 抜け出せる可能性がある | 抜け出せない |

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/local_optimum.png?raw=true" alt="local_optimum" height="200px">

### ミニバッチ勾配降下法 Mini batch gradient descent

　「最急降下法」は、データセット全体をパラメータ更新単位（**バッチ**）としてみなしており、「確率的勾配降下法」は、ランダムに選んだ1サンプルをパラメータ更新単位としてみなす方法です。

　**ミニバッチ勾配降下法（Mini batch gradient descent）**は、上述二つの方法の中間的な方法です。この方法では、ランダムに選んだ複数サンプルを一つのバッチとしてみなして、パラメータの更新をおこないます。

　トレーニングデータのサイズが大きい場合、「最急降下法」ではサンプル全体の計算に時間がかかってしまいますが、「ミニバッチ勾配降下法」では、サブセットを1ユニットとして計算をおこなうため、**解析スピードを損わずにパラメータの更新**をおこなえます。また、「確率的勾配降下法」では1サンプルの影響（外れ値サンプルの影響）を受けやすいですが、「ミニバッチ勾配降下法」は、サブセットの平均値を使うため、**1サンプルの影響を受けにくい**です。加えて、ランダム性も持ち合わせているため、**局所解に陥りにくい**方法です。

## 4. グリッドサーチ Grid search

　機械学習では、**パラメータ（Parameter）**といわれるものがあります。線形回帰の例では、係数$\beta$や誤差$e$がこれに相当します。それらパラメータは、学習によって変動する値であり、学習過程でほぼ最適解（もしくは局所解）が得られるので、機械学習をおこなう者が設定する必要のない値です。
$$y=\beta_1 x_1 + \beta_2 x_2 + ... + \beta_m x_m + e $$

　一方で、**ハイパーパラメータ（Hyperparameter）**と呼ばれる特別なパラメータもあります。このハイパーパラメータは、機械学習をおこなう前に設定し、トレーニング時は更新されない値です。機械学習をおこなう者が任意の値を設定する必要があります。

<small>*※ 今回の実習で登場した「学習率」や「トレーニング回数」はハイパーパラメータです。*</small>

　ハイパーパラメータは、トレーニング結果に大きく影響を与えます。適切な値を設定しなければ、全く意味のない予測モデルができてしまいます。実際の機械学習では、この値の探索（**チューニング Tuning**）が非常に大切です。

　ここでは、良いハイパーパラメータをどうやって見つけるか勉強します。

### グリッドサーチ Grid search

　学習率などの「ハイパーパラメータ」は、学習過程で最適値が求められることはありません。どうやって、ハイパーパラメータを見つけるか？ この答えは、手探りで見つけるしかありません。

　設定可能な値をすべて調べれば、最適なハイパーパラメータを見つけることはできますが、現実的には不可能です。そこで、一定間隔な値のみを調べて、ハイパーパラメータの絞り込みをおこなう、**グリッドサーチ（Grid search）**と呼ばれる方法を用います。

　グリッドサーチには、次の二点が重要です。
- 調べるハイパーパラメータの間隔をどう設定するか
- 何を指標に評価するか

　調べるハイパーパラメータの間隔は、何も情報がない場合、対数的に等間隔な値を使うのが一般的です。例えば、「0.0001, 0.001, 0.01, 0.1, 1.0」といった値です。

　評価指標は、機械学習をおこなう目的によって変わってくるでしょう。良い予測精度をもったモデルを作りたい場合、決定係数$R^2$やラベルの正解率、などを指標に使うことになるでしょう。

　ここでは、「最急降下法」の学習率について、以下の設定値を調べて、最も決定係数$R^2$が高くなる値を探してみましょう。
- 学習回数: 100に固定
- 調べる学習率: 0.0001, 0.001, 0.01, 0.1, 1.0
- 評価指標: 決定係数$R^2$

In [0]:
from sklearn.linear_model import SGDRegressor

# 設定値
n_iter = 100  # 学習回数
eta   = [0.0001, 0.001, 0.01, 0.1, 1.0]   # 学習率

# グリッドサーチ
R2 = []  # 調べた決定係数R2を保存するリスト
for r in eta:
  
  # 「最急降下法」を使った回帰モデルの作成と学習
  model_gd = SGDRegressor(eta0=r, random_state=1, learning_rate="constant")
  for iteration in range(n_iter):
    model_gd.partial_fit(x, y)

  # 学習後、決定係数R2を調べる
  r2 = model_gd.score(x, y)
  R2.append(r2)
  print("eta=", r, " R2=", r2)

 学習率と決定係数の関係をグラフにすると、次のようになります。


In [0]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot(np.log10(eta), R2) # 直線
plt.xlabel("learning rate: log10(eta)")  # x軸ラベル
plt.ylabel("R2", rotation=0)  # y軸ラベル
plt.show()

### 実習2

　例の結果をみると、学習率0.01辺りが最も良さそうです。今度は、その周辺をグリッドサーチして、最適な値を絞り込んでください。

- 学習回数: 100に固定
- 調べる学習率: 0.001から0.1までの値をいくつか設定（例えば、0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.01）
- 評価指標: 決定係数$R^2$

In [0]:
from sklearn.linear_model import SGDRegressor

# --------------- 編集箇所: start ---------------
# 設定値
n_iter = 100  # 学習回数
eta   = []     # 学習率

# --------------- 編集箇所: end -----------------


# グリッドサーチ
R2 = []  # 調べた決定係数R2を保存するリスト
for r in eta:
  
  # 「最急降下法」を使った回帰モデルの作成と学習
  model_gd = SGDRegressor(eta0=r, random_state=1, learning_rate="constant")
  for iteration in range(n_iter):
    model_gd.partial_fit(x, y)

  # 学習後、決定係数R2を調べる
  r2 = model_gd.score(x, y)
  R2.append(r2)
  print("eta=", r, " R2=", r2)

# グラフ
import numpy as np
import matplotlib.pyplot as plt

plt.plot(np.log10(eta), R2) # 直線
plt.xlabel("learning rate: log10(eta)")  # x軸ラベル
plt.ylabel("R2", rotation=0)  # y軸ラベル
plt.show()

#### 解答例

In [0]:
from sklearn.linear_model import SGDRegressor

# --------------- 編集箇所: start ---------------
# 設定値
n_iter = 100  # 学習回数
eta   = [0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1]   # 学習率

# --------------- 編集箇所: end -----------------

# グリッドサーチ
R2 = []  # 調べた決定係数R2を保存するリスト
for r in eta:
  
  # 「最急降下法」を使った回帰モデルの作成と学習
  model_gd = SGDRegressor(eta0=r, random_state=1, learning_rate="constant")
  for iteration in range(n_iter):
    model_gd.partial_fit(x, y)

  # 学習後、決定係数R2を調べる
  r2 = model_gd.score(x, y)
  R2.append(r2)
  print("eta=", r, " R2=", r2)

# グラフ
import numpy as np
import matplotlib.pyplot as plt

plt.plot(np.log10(eta), R2) # 直線
plt.xlabel("learning rate: log10(eta)")  # x軸ラベル
plt.ylabel("R2", rotation=0)  # y軸ラベル
plt.show()

### 検証データ Validation data

　今回は、「トレーニングデータ」と「テストデータ」を分けずに実習してきましたが、機械学習では、通常、データセットを分割して解析します。トレーニングデータは、学習のためのデータセットで、テストデータは、学習結果を評価するためのデータセットです。

　ハイパーパラメータのチューニングやモデル選択をおこないたい場合、**検証データ（Validation data）**というデータも用意するのが一般的です。データセットを「トレーニングデータ」と「検証データ」、「テストデータ」の3つに分割し、「トレーニングデータ」と「検証データ」を使って、ハイパーパラメータのチューニングやモデル選択をおこないます。そして、最も良いパフォーマンスが得られそうなモデルに対して、「テストデータ」を使って、最終的な評価をします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/holdout.png?raw=true" alt="holdout" height="280px">

---
## まとめ

　今回、**勾配法**と呼ばれる最適値探索法を学びました。「勾配法」には、おもに3つの方法、**確率的勾配降下法**、**最急降下法**、**ミニバッチ勾配降下法**があり、そのうち、「確率的勾配降下法」と「最急降下法」の概要と実装方法を学びました。  

　「確率的勾配降下法」は、ランダムに選んだ1サンプルのデータを使ってパラメータを更新していく方法です。「最急降下法」は、データセットのすべてのサンプルに対して、勾配計算をおこない、その平均値でパラメータを更新する方法です。どちらの勾配法にも利点や欠点があります。

　勾配法で有用なモデルを作るためには、**ハイパーパラメータ**のひとつ、**学習率**を適切な値に設定する必要があります。この適切な値を探索する（**チューニング**）方法として、**グリッドサーチ**を学びました。






